In [1]:
import sys
# TO CHANGE
BASEDIR = "../../../../.."
sys.path.insert(0, BASEDIR)

In [2]:
from pprint import pprint
import gc

In [3]:
from src.db_drivers.vector_driver import EmbedderModel, EmbedderModelConfig
from src.kg_model.nodestree_model import NodesTreeModel, NodesTreeModelConfig
from src.agents import AgentDriver, AgentDriverConfig
from src.agents.configs import DEFAULT_AGENT_CONFIGS
from src.utils import TripletCreator, NodeCreator, RelationCreator

/home/dzigen/Desktop/Projects/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Пример работы с NodesTreeModel-частью модели графа знаний

1. Инициализация класса, выполняющего функции embedder-модели

In [4]:
EMBEDDER_MODEL_PATH = '../../../../../models/intfloat/multilingual-e5-small' # PATH TO APPROPRIATE EMBEDDER-MODEL
embedder_config = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH)
pprint(embedder_config)

EmbedderModelConfig(model_name_or_path='../../../../../models/intfloat/multilingual-e5-small',
                    prompts={'passage': 'passage: ', 'query': 'query: '},
                    device='cuda',
                    normalize_embeddings=True)


In [5]:
embedder = EmbedderModel(embedder_config)

No sentence-transformers model found with name ../../../../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [6]:
embedders_mapping = {
    'leaf_dense_nodes': embedder,
    'summ_dense_nodes': embedder
}

2. Инициализация Connector-класса к LLM-моделе

In [7]:
adriver_config = AgentDriverConfig(name='ollama', agent_config=DEFAULT_AGENT_CONFIGS['ollama'])
pprint(adriver_config, width=200)
agent = AgentDriver.connect(adriver_config)

AgentDriverConfig(name='ollama',
                  agent_config=AgentConnectorConfig(gen_strategy={'num_predict': 2048, 'seed': 42, 'temperature': 0.0, 'top_k': 1},
                                                    credentials={'host': 'localhost', 'model': 'llama3.1:8b', 'port': 11437},
                                                    ext_params={'keep_alive': -1, 'timeout': 560}))


In [8]:
output = agent.generate(system_prompt="You are helpful assistant.", user_prompt="What is wrong with human kind?")
print(output[1])
print(output[0])

LLMInferenceStat(prompt_tokens_amount=27, generated_tokens_amount=533, preparation_elapsed_time=0.0, inference_elapsed_time=7.54, timestamp=None)
What a profound and complex question! There's no one-size-fits-all answer, but I'll try to provide some insights.

Humanity has made tremendous progress in various areas, such as science, technology, art, and social justice. However, we also face numerous challenges that hinder our collective well-being and the health of our planet. Here are a few issues that might be contributing to what's "wrong" with humanity:

1. **Self-centeredness and individualism**: Humans often prioritize their own interests over the greater good, leading to conflicts, inequality, and environmental degradation.
2. **Lack of empathy and compassion**: Insufficient understanding and connection with others can lead to prejudice, violence, and social unrest.
3. **Short-sighted decision-making**: Humans tend to focus on short-term gains rather than considering long-term co

3. Инициализация NodesTreeModel-модели

In [9]:
nodestree_config = NodesTreeModelConfig(lang='ru')
pprint(nodestree_config, compact=True)

NodesTreeModelConfig(leafnodes_vdb_driver_configs_mapping={'leaf_bm25_nodes': VectorDriverConfig(db_vendor='inmemory',
                                                                                                 vector_category='sparse_bm25',
                                                                                                 db_config=VectorDBConnectionConfig(db_info={'db': 'default_db',
                                                                                                                                             'table': 'sparsebm25_lnodes'},
                                                                                                                                    params={'load_dump_dir': './personalai_tmp/graph_structures/tree_model/vectorized_leaf_nodes/inmemory_bm25',
                                                                                                                                            'load_from_disk': True,
               

In [10]:
nodestree_model = NodesTreeModel(agent, embedders_mapping, nodestree_config)

In [11]:
nodestree_model.count_items(detailed=True)

{'tree': {'leaf': 0, 'summarized': 0, 'root': 1},
 'vector_leafnodes': {'leaf_dense_nodes': 0, 'leaf_bm25_nodes': 0},
 'vector_summnodes': {'summ_dense_nodes': 0, 'summ_bm25_nodes': 0}}

In [12]:
print("Консистентность модели дерева вершин:", nodestree_model.check_consistency())

Консистентность модели дерева вершин: True


4. Добавление новых вершин в модель дерева вершин

In [13]:
TRIPLET_EXAMPLE1 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='саша'), 
    relation=RelationCreator.create(r_type='simple', name='шла'), 
    end_node=NodeCreator.create(n_type='object', name='улица'))
TRIPLET_EXAMPLE2 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='маша'), 
    relation=RelationCreator.create(r_type='simple', name='шла'), 
    end_node=NodeCreator.create(n_type='object', name='шоссе'))
TRIPLET_EXAMPLE3 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='теплоход'), 
    relation=RelationCreator.create(r_type='simple', name='плыл'), 
    end_node=NodeCreator.create(n_type='object', name='канал'))
TRIPLET_EXAMPLE4 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='моторная лодка'), 
    relation=RelationCreator.create(r_type='simple', name='плыл'), 
    end_node=NodeCreator.create(n_type='object', name='река'))

In [14]:
pprint(TRIPLET_EXAMPLE1, width=200)
pprint(TRIPLET_EXAMPLE2, width=200)
pprint(TRIPLET_EXAMPLE3, width=200)
pprint(TRIPLET_EXAMPLE4, width=200)

Triplet(start_node=Node(name='саша', type=<NodeType.object: 'object'>, prop={}, stringified='саша', id='72adc4fc2f53b1344fb5b56d6ed81b1f'),
        relation=Relation(name='шла', type=<RelationType.simple: 'simple'>, prop={}, id='c2c994676e30850fdf95f0f2a647ba30'),
        end_node=Node(name='улица', type=<NodeType.object: 'object'>, prop={}, stringified='улица', id='2401d7e9c775689eceb9108c20060b0e'),
        stringified='саша шла улица',
        id='9559f35ec745ad1661b65f9847337bc8')
Triplet(start_node=Node(name='маша', type=<NodeType.object: 'object'>, prop={}, stringified='маша', id='d51ca416731c328c6bb159cc30803659'),
        relation=Relation(name='шла', type=<RelationType.simple: 'simple'>, prop={}, id='8c3274995b6a321367f6986fe3f1bace'),
        end_node=Node(name='шоссе', type=<NodeType.object: 'object'>, prop={}, stringified='шоссе', id='956c2d5e231f655cee06efa6a32969e9'),
        stringified='маша шла шоссе',
        id='ad27f4d4f37bfd87612162dceee5d23d')
Triplet(start_node=N

In [15]:
nodestree_model.expand_tree([TRIPLET_EXAMPLE1, TRIPLET_EXAMPLE2, TRIPLET_EXAMPLE3, TRIPLET_EXAMPLE4])
print("Количество объектов в древовидной моделе хранения вершин: ")
pprint(nodestree_model.count_items(detailed=True))
print("Консистентность модели дерева вершин:", nodestree_model.check_consistency())

100%|██████████| 8/8 [00:30<00:00,  3.84s/it]

Количество объектов в древовидной моделе хранения вершин: 
{'tree': {'leaf': 8, 'root': 1, 'summarized': 7},
 'vector_leafnodes': {'leaf_bm25_nodes': 8, 'leaf_dense_nodes': 8},
 'vector_summnodes': {'summ_bm25_nodes': 7, 'summ_dense_nodes': 7}}
Консистентность модели дерева вершин: True


5. Сопоставление сущности с вершинами из соответствующей древовидной модели

In [16]:
ENTITE_EXAMPLE1 = "олеся"
ENTITE_EXAMPLE2 = "крейсер"
ENTITE_EXAMPLE3 = "проезжая чать"

In [17]:
matched_nodes1 = nodestree_model.match_entitie2objects(ENTITE_EXAMPLE1)
print("query_entitie: ", ENTITE_EXAMPLE1)
print("matched nodes: ", list(map(lambda inst: inst.document, matched_nodes1)))

matched_nodes2 = nodestree_model.match_entitie2objects(ENTITE_EXAMPLE2)
print("query_entitie: ", ENTITE_EXAMPLE2)
print("matched nodes: ", list(map(lambda inst: inst.document, matched_nodes2)))

matched_nodes3 = nodestree_model.match_entitie2objects(ENTITE_EXAMPLE3)
print("query_entitie: ", ENTITE_EXAMPLE3)
print("matched nodes: ", list(map(lambda inst: inst.document, matched_nodes3)))

nodestree_model.count_items()
nodestree_model.check_consistency()

query_entitie:  олеся
matched nodes:  ['саша']
query_entitie:  крейсер
matched nodes:  ['теплоход']
query_entitie:  проезжая чать
matched nodes:  ['канал']


True

6. Полное удаление содержимого модели дерева вершин

In [18]:
nodestree_model.clear()
nodestree_model.count_items(detailed=True)

{'tree': {'leaf': 0, 'summarized': 0, 'root': 1},
 'vector_leafnodes': {'leaf_dense_nodes': 0, 'leaf_bm25_nodes': 0},
 'vector_summnodes': {'summ_dense_nodes': 0, 'summ_bm25_nodes': 0}}

In [ ]:
del nodestree_model 
gc.collect()